# DealInfos_Word2Vec feature를 ES에 벌크 인서트

-  환경: ElasticSearch 6
-  테스트 머신: 장민 PC

```
PUT /dealinfos
{
mapping_dealinfos_word2vec.json
}
```


In [1]:
import requests
import time
import urllib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import json
import elasticsearch5
import csv
import pickle
from elasticsearch5.helpers import bulk
import re
import glob
import os
from datetime import timezone, timedelta, datetime
from pymongo import MongoClient

In [8]:
def make_documents(file_path):
    pred_dic = {}
    with open(file_path, 'rb') as jsonfile:
        for row in jsonfile:
            source_ = eval(row)
            source = {}
            source['v'] = source_['v']
            source['words'] = source_['words']
            source['values'] = source_['vectorizedWords']['values']
            doc =  {
                "_op_type":'create',
                "_index":"deal_word2vec",
                "_type":"doc",
                "_id": source['v'],
                "_source": source
            }
            yield(doc)

In [3]:
### Jangmin's PC
es_url = 'localhost:9200'

In [4]:
es = elasticsearch5.Elasticsearch(es_url)

In [5]:
es.indices.get_settings(index='deal_word2vec')

{'deal_word2vec': {'settings': {'index': {'creation_date': '1528197997269',
    'number_of_replicas': '1',
    'number_of_shards': '5',
    'provided_name': 'deal_word2vec',
    'uuid': 'dDBs71HjRxKkC48L-SoMCA',
    'version': {'created': '6020499'}}}}}

# bulk inserting to ES

In [13]:
data_dir = r'd:\WMIND\temp\w2vec.json'
start = time.time()
total_time = 0
bulk(es, make_documents(data_dir))
total_time += (time.time() - start)
print("Elapsed Time {} sec".format(total_time))

Elapsed Time 1436.8922922611237 sec


In [16]:
res = es.search(index='deal_word2vec', body={'query': {'match':{'v':'2668602'}}})

In [17]:
res['hits']['total']

1

In [18]:
res


{'_shards': {'failed': 0, 'skipped': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '2668602',
    '_index': 'deal_word2vec',
    '_score': 1.0,
    '_source': {'v': 2668602,
     'values': [-1.7802355289459229,
      18.870119285583495,
      2.5163070142269137,
      -10.703756618499757,
      1.600479006767273,
      3.5656559824943543,
      -5.844938635826111,
      6.633014583587647,
      8.395471286773683,
      -23.7651406288147,
      14.762410998344421,
      10.690511178970338,
      8.746411949396133,
      28.675686740875246,
      8.49290874004364,
      -3.034431028366089,
      36.52057704925537,
      38.59238967895508,
      -3.684904098510742,
      33.746253967285156,
      -8.964104914665223,
      -15.591403150558472,
      -0.33317070007324223,
      30.87879137992859,
      9.32277364730835,
      29.246194028854372,
      -28.27330641746521,
      -20.553472375869752,
      -11.5706449508667,
      -4.212529945373535,
      -10.324701023101808,
  